#### Задача №1
Дана матрица объект-признак X и значения целевой переменной y.
Подберите два признака (из четырёх) так, чтобы уровень линейной зависимости целевой переменной от значений этих признаков был максимальным. Другими словами, модель линейной регрессии на этих признаках должна давать наилучший результат. В качестве ответа запишите значение коэффициента детерминации для модели на этих признаках.

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from itertools import combinations

Ncomb = list(combinations('0123', 2)) # список сочетаний номеров столбцов (2 из 4)

X0 = np.array([[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
             [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
             [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
             [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
             [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
             [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
             [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
             [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
             [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
             [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]])
y0 = np.array([ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
              7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218])

R2max=0
for i in Ncomb: # перебираем все возможные сочетания 2-х из 4-х столбцов
    X1=np.vstack([X0[:,int(i[0])],X0[:,int(i[1])]]).T # "собираем" матрицу из двух признаков и транспонируем ее
    ones = np.ones((X1.shape[0], 1))
    X1 = np.hstack((ones, X1)) # добавляем столбец едениц, для использования МНК
    lr = LinearRegression(fit_intercept=False).fit(X1, y0) # строим линейную регресию
    E = y0 - lr.coef_[0] - lr.coef_[1]*X1[:,1] - lr.coef_[2]*X1[:,2] # вычисляем ошибки
    SSy=((y0-y0.mean())**2).sum() # сумма квадратов отклонений значений от их среднего
    SSres=((E-E.mean())**2).sum() # сумма квадратов отклонений ошибок от их среднего
    R2= 1-SSres/SSy # коэффициент детерминации
    if R2>R2max:
        R2max=R2
        Best_X = i

print(f'максимальный R2={round(R2max,8)} при использовании признаков №{int(Best_X[0])+1} и №{int(Best_X[1])+1}')
    


максимальный R2=0.76342462 при использовании признаков №1 и №3


#### Задача 2
Является ли значимым уравнение регрессии, полученное в предыдущей задаче? В качестве ответа запишите 1, если является, и 0 иначе.

In [12]:
from scipy import stats
X1=np.vstack([X0[:,0],X0[:,2]]).T
n = X1.shape[0]
m = 2

k1 = m
k2 = n - m - 1

alpha = 0.05

t = stats.f.ppf(1 - alpha, k1, k2)
F = (R2max / k1) / ((1 - R2max) / k2)
print(f"Cтатистика F={F} больше левой границы критической области t={t}")
print(f"т.е. согласно тесту Фишера нулевая гипотеза о НЕзначимости уравнения регрессии отвергается - уравнение значимо.")
print(f"Ответ: 1")


Cтатистика F=11.29443912292265 больше левой границы критической области t=4.73741412777588
т.е. согласно тесту Фишера нулевая гипотеза о НЕзначимости уравнения регрессии отвергается - уравнение значимо.
Ответ: 1


#### Задача 3
Для проведения A/B-тестирования сайта интернет-магазина были получены следующие данные: страница `A` была посещена 2509 раз, из них 77 закончились совершением покупки, страница `B` была посещена 1465 раз, 60 из них закончились совершением покупки. Является ли значимым отличие конверсии на страницах `A` и `B`? В качестве ответа запишите 1, если является, и 0 иначе.

In [39]:
n1 = 2509
n2 = 1465
M1 = 77/n1 # выборочная конверсия A, аналог выборочного среднего
M2 = 60/n2 # выборочная конверсия B

s1 = ((77*(1-M1)**2+(n1-77)*M1**2)/(n1-1))**0.5 # стандартное отклонение при 77 результаттах "1" и остальных "0"
s2 = ((60*(1-M2)**2+(n2-77)*M2**2)/(n2-1))**0.5 # аналогично, для B случая

s_delta = np.sqrt(s1 ** 2 / n1 + s2 ** 2 / n2)
t = (M1 - M2) / s_delta
df = (s1 ** 2 / n1 + s2 ** 2 / n2) ** 2 /  ((s1 ** 2 / n1) ** 2 / (n1 - 1) + (s2 ** 2 / n2) ** 2 / (n2 - 1))

alpha = 0.05

t1 = stats.t.ppf(alpha / 2, df=df)
t2 = stats.t.ppf(1 - alpha / 2, df=df)

print (f"Статистика t={round(t,2)} попадает в доверительный интервал {round(t1,2), round(t2,2)}")
print (f"Значит нулевая гипотеза о совпадении среднего (конверсии) в двух выборках верна.")
print (f"Другими словами ОТЛИЧИЕ конверсии НЕ является значимым.")
print (f"Ответ: 0")

Статистика t=-1.65 попадает в доверительный интервал (-1.96, 1.96)
Значит нулевая гипотеза о совпадении среднего (конверсии) в двух выборках верна.
Другими словами ОТЛИЧИЕ конверсии НЕ является значимым.
Ответ: 0
